In [3]:
%%capture
!python -m pip install --upgrade pip
!pip install -r requirements.txt

In [5]:
from tqdm import tqdm
import json

import torch
import math

from transformers import AutoTokenizer
from datasets import load_dataset

from trainer.SFTTrainer import train
from model.args import MOEModelArgs
from model.KANaMoEv1 import KANaMoEv1

from utils import load_model, quick_inference

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def lr_lambda(current_step: int, max_steps: int=50000, warmup_steps: int=40, lr_scheduler_type: str="cosine"):
    if current_step < warmup_steps:
        return current_step / warmup_steps

    annealing_steps = max_steps - warmup_steps

    if annealing_steps <= 0:
        annealing_steps = 1

    progress = (current_step - warmup_steps) / annealing_steps
    if lr_scheduler_type == "cosine":
        new_learning_rate = 0.5 * (1.0 + math.cos(math.pi * progress))
    elif lr_scheduler_type == "sinus":
        new_learning_rate = 0.5 * (1.0 + math.sin(math.pi * progress))
    else:
        new_learning_rate = 1.0
    return new_learning_rate

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")
print("[TOKENIZER LOADED]")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


MOEModelArgs.vocab_size = tokenizer.vocab_size
MOEModelArgs.pad_id = tokenizer.pad_token_id
MOEModelArgs.num_experts_per_tok = 4
MOEModelArgs.max_batch_size = 100
MOEModelArgs.max_seq_len = 128
MOEModelArgs.num_experts = 14
MOEModelArgs.n_layers = 18
MOEModelArgs.dim = 256
MOEModelArgs.n_heads = 12
MOEModelArgs.n_kv_heads = 6
# MOEModelArgs.use_kan = False
# MOEModelArgs.use_softmax_temp_proj = False

In [ ]:
# Load the dataset (replace the old one)
# use the new dataset "sample-10BT" (adjust as needed)
# dataset = load_dataset("HuggingFaceFW/fineweb", name="CC-MAIN-2024-18", split="train", streaming=True)
# BEE-spoke-data/fineweb-1000_64k
# BEE-spoke-data/fineweb-100_128k
# pszemraj/fineweb-1k_long

dataset = load_dataset("BEE-spoke-data/fineweb-1000_64k", split="train", streaming=True)

# List to hold all tokenized data
tokenized_data = []

# Process the dataset
print("\nProcessing dataset:")
for data in tqdm(dataset, desc="Tokenizing dataset"):
    # Tokenize the current entry without truncation or padding
    tokens = tokenizer(data['text'], truncation=False, padding=False)['input_ids']
    # Add the tokenized IDs to the list
    tokenized_data.append(torch.tensor(tokens))

# Concatenate all tokenized sequences into a single tensor
data = torch.cat(tokenized_data, dim=0).unsqueeze(0)  # unsqueeze to add a batch dimension

# Define train/val split based on the size of the dataset
n = int(0.9 * data.size(1))  # data.size(1) because sequences are concatenated along dimension 1
train_data = data[:, :n].to(device)
val_data = data[:, n:].to(device)

In [ ]:
print("\n[LOADING MODEL]\n")
model = KANamav5(MOEModelArgs, device=device)

In [ ]:
# Starting sequence (as tokens)
initial_text = "Once upon a time"
initial_tokens = tokenizer(initial_text, return_tensors="pt").input_ids.to(device)

# Perform inference
generated_tokens, generated_text = quick_inference(model, initial_tokens, max_new_tokens=50, tokenizer=tokenizer)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

print("\n[TRAINING MODEL]\n")
new_model = train(
    model=model,
    optimizer=optimizer,
    train_data=train_data,
    val_data=val_data,
    scheduler=scheduler,
    save_model_name="KANama-medium",
    max_steps=100000,
    loss_interval=5,
    eval_interval=50,
    device=device
)